<a href="https://colab.research.google.com/github/naeemamr/Project-Allocation-Recommender-System/blob/main/Thesis_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing all Libraries
!pip install mlxtend
!pip install scikit-surprise==1.1.1
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp310-cp310-linux_x86_64.whl size=2051683 sha256=6bb47e7937a82db8b6d8ce24fc0a958d62d62f250065e57bdd14637120e864f8
  Stored in directory: /root/.cache/pip/wheels/83/11/fb/6f834df4a72a79fcd80a6ab5e3098cee14dc66bda025915ace
Successfully built scikit-surprise


##Human Resources Model
###Business Understanding:
This model uses a collection of datasets that were provided for IT employee data for project allocation. We identified three important datasets that were relevant for building our recommender system, which is designed to analyze IT projects, extract related features, and provide employee recommendations for the best fit for a given project. The datasets we used include Employee_Skills_Dataset.csv, Final_Employee_Data.csv, and Project_details.csv, as they contained the most relevant features to address our business problem.
The goal of this recommender system is to help businesses optimize their project allocation and improve resource utilization by analyzing the skills and experience of individual employees.

###Data Understanding:
**Employee_Skills_Dataset.csv**
This dataset contains information about the skill levels of IT employees in various programming languages and technologies. The skill levels are represented on a scale of 0 to 4, where 0 indicates no skill and 4 indicates a high level of skill. The dataset includes the following columns:

    Eid: A unique identifier for each employee.
    Python: The skill level of the employee in Python programming language.
    Machine Learning: The skill level of the employee in Machine Learning.
    Deep Learning: The skill level of the employee in Deep Learning.
    Data Analysis: The skill level of the employee in Data Analysis.
    Asp.Net: The skill level of the employee in the Asp.Net web development framework.
    Ado.Net: The skill level of the employee in Ado.Net.
    VB.Net: The skill level of the employee in VB.Net programming language.
    C#: The skill level of the employee in C# programming language.
    Java: The skill level of the employee in Java programming language.
    Spring Boot: The skill level of the employee in the Spring Boot framework.
    Hibernate: The skill level of the employee in the Hibernate ORM framework.
    NLP: The skill level of the employee in Natural Language Processing techniques.
    CV: The skill level of the employee in Computer Vision techniques.
    JS: The skill level of the employee in JavaScript programming language.
    React: The skill level of the employee in the React JavaScript library.
    Node: The skill level of the employee in the Node.js JavaScript runtime.
    Angular: The skill level of the employee in the Angular framework.
    Dart: The skill level of the employee in Dart programming language.
    Flutter: The skill level of the employee in the Flutter mobile app development framework.
    Vb.Net: The skill level of the employee in the VB.Net programming language.
**Final_Employees_Data.csv**
This dataset contains information about IT employees, including their       experience, total number of projects, rating, areas of interest, programming languages, and project counts.

    Eid: A unique identifier for each employee.
    Ename: The name of the employee.
    Experience: The total number of years of experience the employee has in the IT industry.
    Total_projects: The total number of projects the employee has worked on.
    Rating: The employee's rating based on their performance.
    Area_of_Interest_1, Area_of_Interest_2, Area_of_Interest_3: The areas of interest or expertise of the employee, with up to three options listed.
    Language1, Language2, Language3: The programming languages with which the employee has experience, with up to three options listed.
    AI_project_count, ML_project_count, JS_project_count, Java_project_count, DotNet_project_count, Mobile_project_count: The number of projects the employee has worked on in each of these areas, grouped together for ease of understanding.
**Project_details.csv**
Originaly this dataset included these columns.

    Project Name: This column contains the name  of the ptrvious projects.
    Client Name: This column contains the ID of the client for whom the project is being executed.
    Domain Name: This column contains the domain to which the project belongs.
However, Since there were no actual use for this dataset but to indicate the experiences required for each project's domain. So we edited the datasest to include more 3 more projects to include more domains.    



Data Exploration and Cleaning:

We'll read in *Employee_Skills_Datset.xls* dataset:

In [ ]:
df_skills = pd.read_csv("/Employee_Skills_Datset.xls")

In [ ]:
df_skills.head()

,Eid,Python,Machine Learning,Deep Learning,Data Analysis,Asp.Net,Ado.Net,VB.Net,C#,Java,...,Hibernate,NLP,CV,JS,React,Node,Angular,Dart,Flutter,Vb.Net
0,1,1,1,1,1,1,1,0,1,1,...,0,1,1,3,3,3,3,0,0,1
1,2,1,1,1,1,4,4,0,4,1,...,1,3,3,1,1,1,1,1,1,4
2,3,4,4,4,4,1,1,0,1,2,...,2,1,1,3,3,3,3,1,1,1
3,4,3,3,3,3,4,4,0,4,1,...,1,1,1,1,1,1,1,2,2,4
4,5,1,1,1,1,1,1,0,1,3,...,3,1,1,2,2,2,2,4,4,1


In [ ]:
df_skills.columns

Index(['Eid', 'Python', 'Machine Learning', 'Deep Learning', 'Data Analysis',
       'Asp.Net', 'Ado.Net', 'VB.Net', 'C#', 'Java', 'Spring Boot',
       'Hibernate', 'NLP', 'CV', 'JS', 'React', 'Node', 'Angular', 'Dart',
       'Flutter', 'Vb.Net'],
      dtype='object')

In [ ]:
#We'll identify the unique values in the dataset
for column in df_skills.columns:
    print(f"{column} : {df_skills[column].unique()}")
    print("====================================")
#No anomalies or NaN values and all dtypes are of them same type, so everything seems fine.    

Eid : [   1    2    3    4    5    6    7    8    9   10   11   12   13   14
   15   16   17   18   19   20   21   22   23   24   25   26   27   28
   29   30   31   32   33   34   35   36   37   38   39   40   41   42
   43   44   45   46   47   48   49   50   51   52   53   54   55   56
   57   58   59   60   61   62   63   64   65   66   67   68   69   70
   71   72   73   74   75   76   77   78   79   80   81   82   83   84
   85   86   87   88   89   90   91   92   93   94   95   96   97   98
   99  100  101  102  103  104  105  106  107  108  109  110  111  112
  113  114  115  116  117  118  119  120  121  122  123  124  125  126
  127  128  129  130  131  132  133  134  135  136  137  138  139  140
  141  142  143  144  145  146  147  148  149  150  151  152  153  154
  155  156  157  158  159  160  161  162  163  164  165  166  167  168
  169  170  171  172  173  174  175  176  177  178  179  180  181  182
  183  184  185  186  187  188  189  190  191  192  193  194  195  196


In [ ]:
#To identify the number of unique values
for column in df_skills.columns:
    print(f"{column}: Number of unique values {df_skills[column].nunique()}")
    print("==========================================================")

Eid: Number of unique values 1000
Python: Number of unique values 4
Machine Learning: Number of unique values 5
Deep Learning: Number of unique values 5
Data Analysis: Number of unique values 5
Asp.Net: Number of unique values 5
Ado.Net: Number of unique values 5
VB.Net: Number of unique values 1
C#: Number of unique values 4
Java: Number of unique values 4
Spring Boot: Number of unique values 5
Hibernate: Number of unique values 5
NLP: Number of unique values 5
CV: Number of unique values 5
JS: Number of unique values 4
React: Number of unique values 5
Node: Number of unique values 5
Angular: Number of unique values 5
Dart: Number of unique values 5
Flutter: Number of unique values 5
Vb.Net: Number of unique values 5


In [ ]:
df_skills.isnull().sum()
#Yup, no null values

Eid                 0
Python              0
Machine Learning    0
Deep Learning       0
Data Analysis       0
Asp.Net             0
Ado.Net             0
VB.Net              0
C#                  0
Java                0
Spring Boot         0
Hibernate           0
NLP                 0
CV                  0
JS                  0
React               0
Node                0
Angular             0
Dart                0
Flutter             0
Vb.Net              0
dtype: int64

In [ ]:
df_skills.dtypes 
#All of the same type too.

Eid                 int64
Python              int64
Machine Learning    int64
Deep Learning       int64
Data Analysis       int64
Asp.Net             int64
Ado.Net             int64
VB.Net              int64
C#                  int64
Java                int64
Spring Boot         int64
Hibernate           int64
NLP                 int64
CV                  int64
JS                  int64
React               int64
Node                int64
Angular             int64
Dart                int64
Flutter             int64
Vb.Net              int64
dtype: object

We'll read in *Final_Employees_Data.xls* dataset:

In [ ]:
df_emp = pd.read_csv("/Final_Employees_Data.xls")

In [ ]:
df_emp.head()

,Eid,Ename,Experience,Total_projects,Rating,Area_of_Interest_1,Area_of_Interest_2,Area_of_Interest_3,Language1,Language2,Language3,AI_project_count,ML_project_count,JS_project_count,Java_project_count,DotNet_project_count,Mobile_project_count
0,1,Dana Bond,2,2,1.000000,DotNet,JS,AI,English,Hindi,Tamil,0,1,1,0,0,0
1,2,Jesus Hampton,9,13,1.444444,DotNet,AI,JS,English,Hindi,Malayalam,4,3,0,2,3,1
2,3,Teresa Munoz,15,16,1.066667,ML,JS,Java,English,Kannada,Chinese,3,3,1,3,5,1
3,4,Annette Dipietro,18,19,1.055556,DotNet,ML,Mobile,English,Chinese,Tamil,4,4,2,3,3,3
4,5,Jennifer Fortner,22,15,0.681818,Mobile,Java,JS,English,Chinese,Telugu,3,4,3,1,1,3


In [ ]:
df_emp.isnull().sum()
#Also, no null values.

Eid                     0
Ename                   0
Experience              0
Total_projects          0
Rating                  0
Area_of_Interest_1      0
Area_of_Interest_2      0
Area_of_Interest_3      0
Language1               0
Language2               0
Language3               0
AI_project_count        0
ML_project_count        0
JS_project_count        0
Java_project_count      0
DotNet_project_count    0
Mobile_project_count    0
dtype: int64

In [ ]:
#We'll identify the unique values in the dataset
for column in df_emp.columns:
    print(f"{column} : {df_emp[column].unique()}")
    print("====================================")
#No null values or anomalies in this dataset too    

Eid : [   1    2    3    4    5    6    7    8    9   10   11   12   13   14
   15   16   17   18   19   20   21   22   23   24   25   26   27   28
   29   30   31   32   33   34   35   36   37   38   39   40   41   42
   43   44   45   46   47   48   49   50   51   52   53   54   55   56
   57   58   59   60   61   62   63   64   65   66   67   68   69   70
   71   72   73   74   75   76   77   78   79   80   81   82   83   84
   85   86   87   88   89   90   91   92   93   94   95   96   97   98
   99  100  101  102  103  104  105  106  107  108  109  110  111  112
  113  114  115  116  117  118  119  120  121  122  123  124  125  126
  127  128  129  130  131  132  133  134  135  136  137  138  139  140
  141  142  143  144  145  146  147  148  149  150  151  152  153  154
  155  156  157  158  159  160  161  162  163  164  165  166  167  168
  169  170  171  172  173  174  175  176  177  178  179  180  181  182
  183  184  185  186  187  188  189  190  191  192  193  194  195  196


In [ ]:
#To identify the number of unique values
for column in df_emp.columns:
    print(f"{column}: Number of unique values {df_emp[column].nunique()}")
    print("==========================================================")

Eid: Number of unique values 1000
Ename: Number of unique values 1000
Experience: Number of unique values 23
Total_projects: Number of unique values 28
Rating: Number of unique values 197
Area_of_Interest_1: Number of unique values 6
Area_of_Interest_2: Number of unique values 6
Area_of_Interest_3: Number of unique values 6
Language1: Number of unique values 1
Language2: Number of unique values 7
Language3: Number of unique values 7
AI_project_count: Number of unique values 12
ML_project_count: Number of unique values 11
JS_project_count: Number of unique values 12
Java_project_count: Number of unique values 12
DotNet_project_count: Number of unique values 10
Mobile_project_count: Number of unique values 11


In [ ]:
df_emp.dtypes

Eid                       int64
Ename                    object
Experience                int64
Total_projects            int64
Rating                  float64
Area_of_Interest_1       object
Area_of_Interest_2       object
Area_of_Interest_3       object
Language1                object
Language2                object
Language3                object
AI_project_count          int64
ML_project_count          int64
JS_project_count          int64
Java_project_count        int64
DotNet_project_count      int64
Mobile_project_count      int64
dtype: object

Now we'll read in Project_details.xlsx:

In [ ]:
import xlrd
df_proj = pd.read_excel("/Project_details.xlsx", engine='openpyxl')

In [ ]:
df_proj.head()
#We won't need to identify the unique variables anyways.
#Note: We added more data in this dataset to include more Domain Names to utilize all skills in df_skills

,Project Name,Client Name,Domain Name
0,Web application,XYZ,JS Project
1,AI based application for web tracking,TCS,ML Project
2,Covid Tracking,HITS,AI Project
3,Mobile application,NGO,Mobile Project
4,DotNet,HSBC,DotNet Project


In [ ]:
#We'll create these columns in df_proj and set them with 0s representing any skill a project could require
df_proj[['Python', 'Machine Learning', 'Deep Learning', 'Data Analysis',
         'Asp.Net', 'Ado.Net', 'VB.Net', 'C#', 'Java', 'Spring Boot',
         'Hibernate', 'NLP', 'CV', 'JS', 'React', 'Node', 'Angular', 'Dart',
         'Flutter', 'Vb.Net']] = 0

In [ ]:
#We then created a loop code that iterates through each row in the df_proj to set the appropriate skills column value with 1 indicating it requires that skill
for i in range(len(df_proj)):
      if df_proj.loc[i, 'Domain Name'] == 'AI Project':
          df_proj.loc[i, ['Python', 'Machine Learning', 'Deep Learning', 'NLP', 'CV']] = 1
      elif df_proj.loc[i, 'Domain Name'] == 'ML Project':
          df_proj.loc[i, ['Python', 'Machine Learning', 'Deep Learning', 'Data Analysis', 'NLP', 'CV']] = 1
      elif df_proj.loc[i, 'Domain Name'] == 'DotNet Project':
          df_proj.loc[i, ['Asp.Net', 'Ado.Net', 'VB.Net', 'C#']] = 1
      elif df_proj.loc[i, 'Domain Name'] == 'Java Project':
          df_proj.loc[i, ['Java', 'Spring Boot', 'Hibernate']] = 1
      elif df_proj.loc[i, 'Domain Name'] == 'JS Project':
          df_proj.loc[i, ['JS', 'React', 'Node', 'Angular']] = 1
      elif df_proj.loc[i, 'Domain Name'] == 'Mobile Project':
          df_proj.loc[i, ['Dart', 'Flutter', 'React']] = 1
#We researched on which skills from df_skills that are utilized in each domain and this was what we reached.          

In [ ]:
#Here we defined a dictionary that maps the type of a project to the skills required
domain_skills = { 'AI Project': ['Python', 'Machine Learning', 'Deep Learning', 'NLP', 'CV'],
                  'ML Project': ['Python', 'Machine Learning', 'Deep Learning','Data Analysis'],
                  'JS Project': ['JS', 'React', 'Node', 'Angular'],
                  'Java Project': ['Java', 'Spring Boot', 'Hibernate'], 
                  'DotNet Project': ['Asp.Net', 'Ado.Net', 'VB.Net', 'C#', 'Vb.Net'], 
                  'Mobile Project': ['Dart', 'Flutter'] }

In [ ]:
#Now we'll define a function that takes as input a domain name and returns the required skills for the domain using the dictionary
def get_required_skills(domain_name):
    return domain_skills[domain_name]

In [ ]:
#Here we'll define the target project that the recommendations will be based on
target_project = 'Java Project'

#We used get_required skills function to create an array of the required skills for that project 
required_skills = get_required_skills(target_project)

In [ ]:
required_skills

['Java', 'Spring Boot', 'Hibernate']

Collaborative Filtering Model:

In [ ]:
#We added Experience and Rating of each employee as its useful for recommendations
df_skills = df_skills.merge(df_emp[['Eid', 'Experience', 'Rating']], on='Eid', how='left')

Now **df_skills** contains Eid, all skills columns, Experience and Rating; all numerical and considered rating for the column it represents.

In [ ]:
#Creating user-item matrix for Collaborative Filtering
df_user_item = pd.pivot_table(df_skills, index='Eid')

The columns in this user-item matrix correspond to different skills and each employee's experience/rating so that we could quantitatively analyze and compare the skill levels of employees.

In [ ]:
df_user_item

,Ado.Net,Angular,Asp.Net,C#,CV,Dart,Data Analysis,Deep Learning,Experience,Flutter,...,Java,Machine Learning,NLP,Node,Python,Rating,React,Spring Boot,VB.Net,Vb.Net
Eid,,,,,,,,,,,,,,,,,,,,,
1,1,3,1,1,1,0,1,1,2,0,...,1,1,1,3,1,1.000000,3,0,0,1
2,4,1,4,4,3,1,1,1,9,1,...,1,1,3,1,1,1.444444,1,1,0,4
3,1,3,1,1,1,1,4,4,15,1,...,2,4,1,3,4,1.066667,3,2,0,1
4,4,1,4,4,1,2,3,3,18,2,...,1,3,1,1,3,1.055556,1,1,0,4
5,1,2,1,1,1,4,1,1,22,4,...,3,1,1,2,1,0.681818,2,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,1,1,1,1,2,3,1,1,24,3,...,4,1,2,1,1,1.125000,1,4,0,1
997,2,1,2,2,1,1,1,1,3,1,...,1,1,1,1,1,2.000000,1,1,0,2
998,0,1,0,1,4,1,3,3,14,1,...,1,3,4,1,3,0.357143,1,1,0,0


In [ ]:
#Since all our included columns are numerical so it will be much easier to standarize all values
scaler = StandardScaler()
df_user_item_scaled = scaler.fit_transform(df_user_item)
df_user_item_scaled = pd.DataFrame(df_user_item_scaled, index=df_user_item.index, columns=df_user_item.columns)

In [ ]:
#Filtering the user-item matrix to only include employees with the required skills and drop employees with 0s in the required skills array
relevant_employees = df_user_item_scaled.loc[(df_skills[required_skills] != 0).all(axis=1).values]

In [ ]:
#Merge with df_emp to include other relevant employee details
df_emp_skills = pd.merge(relevant_employees, df_emp, on='Eid', how='left')

In [ ]:
df_emp_skills = df_emp_skills.drop(['Experience_y', 'Rating_y'], axis=1)
df_emp_skills = df_emp_skills.rename(columns={'Experience_x': 'Experience', 'Rating_x': 'Rating'})

In [ ]:
#This code is generated and we provided explaination of the code below
#Join with df_proj to add the Domain Name of the project and include only the columns relevant to the Domain Name
df_proj_relevant = df_proj.loc[:, required_skills + ['Domain Name']]
df_emp_proj_skills = df_emp_skills[['Eid', 'Ename', 'Experience', 'Total_projects', 'Rating'] + required_skills]
df_emp_proj_skills = pd.merge(df_emp_proj_skills, df_proj_relevant, on=required_skills, how='left')
df_emp_proj_skills['Domain Name'] = ['Java Project']* len(df_emp_proj_skills)

#Explanation:
# - Select the relevant project columns based on required skills and 'Domain Name'
# - Select specific employee columns along with the required skills
# - Merge the employee skills DataFrame with the relevant project DataFrame based on required skills
# - Assign 'Java Project' as the value for the 'Domain Name' column for all rows in the merged DataFrame

#Select only the relevant columns
df_emp_proj_skills = df_emp_proj_skills[['Eid', 'Ename', 'Experience', 'Total_projects', 'Rating', 'Domain Name'] + required_skills]

<ipython-input-19-a46ea0c348e0>:5: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  df_emp_proj_skills = pd.merge(df_emp_proj_skills, df_proj_relevant, on=required_skills, how='left')


In [ ]:
#We'll create a reader object tp parse the rating data and to read the dataframe and prepare it for Collaborative Filtering
#We used relevant_employees to determine the min and max values of rating scale
reader = Reader(rating_scale=(relevant_employees.values.min(), relevant_employees.values.max()))

In [ ]:
#We'll use Singualar Value Decompisition(SVD) as its the best one that worked with datasets
#We'll set biased with false to ensure that the ratings are generated solely from the user-item matrix
algo = SVD(biased=False, verbose=False, random_state=42)

Collaborative Filtering Model Recommendations:

In [ ]:
#Create a train dataframe and convert it into a surprise dataset
train_df = pd.melt(df_skills, id_vars=['Eid'], value_vars=required_skills, var_name='Skills', value_name='Rating')
data = Dataset.load_from_df(train_df[['Eid', 'Skills', 'Rating']], reader)

#Train the algorithm on the entire dataset
trainset = data.build_full_trainset()
algo.fit(trainset)
#During the training process, the SVD algorithm analyzes the provided dataset to learn the underlying patterns and relationships between users, skills, and ratings.
#It decomposes the original rating matrix into three matrices: U, Σ, and V^T. U represents user features, Σ represents the singular values, and V^T represents skill features.

<ipython-input-22-0b8c6af77e27>:2: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  train_df = pd.melt(df_skills, id_vars=['Eid'], value_vars=required_skills, var_name='Skills', value_name='Rating')


In [ ]:
#create a list for all possible employee-skill combinations
emp_list = relevant_employees.index.tolist()
skill_list = required_skills

#Here we created a tuple for each combination of an employee from the emp_list and a skill from the skill_list
testset = [(e, s, None) for e in emp_list for s in skill_list]

#Now we'll get the prediction ratings for all employee-skill combinations
predictions = algo.test(testset)

#And convert it into a dataframe 
pred_df = pd.DataFrame([[p.uid, p.iid, p.est] for p in predictions], columns=['Eid', 'Skills', 'Estimated_Rating'])

In [ ]:
#Filter the dataframe to only include employees with the required skills
relevant_pred_df = pred_df.loc[pred_df['Skills'].isin(required_skills)]

#Sort the dataframe in descending order based on the Estimated_Rating column
relevant_pred_df = relevant_pred_df.sort_values(by=['Estimated_Rating'], ascending=False)

#Get the top 5 employees
top_5_employees = relevant_pred_df['Eid'].unique()[:5]

#Print the top 5 employees
for employee in top_5_employees:
    print('Employee:', employee)
 

Employee: 914
Employee: 833
Employee: 996
Employee: 990
Employee: 933


Collaborative Filtering Model Evaluation

In [ ]:
#Create a different train set to avoid overfitting
df = pd.melt(df_skills, id_vars=['Eid'], value_vars=required_skills, var_name='Skills', value_name='Rating')
data = Dataset.load_from_df(df[['Eid', 'Skills', 'Rating']], reader)

<ipython-input-25-2d4cb3fa7f79>:2: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  df = pd.melt(df_skills, id_vars=['Eid'], value_vars=required_skills, var_name='Skills', value_name='Rating')


In [ ]:
from surprise.model_selection import KFold

In [ ]:
#Since we have a relatively small dataset we'll use K-fold cross-validation to evaluate the algorithm
kf = KFold(n_splits=5, random_state=42)

In [ ]:
#Evaluate the performance of the algorithm using RMSE and MSE metrics
rmse_list = []
mse_list = []
mae_list = []
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    test_pred = algo.test(testset)
    pred = np.array([p.est for p in test_pred])
    test_matrix = np.array([p.r_ui for p in test_pred])
    rmse = sqrt(mean_squared_error(test_matrix, pred))
    mse = mean_squared_error(test_matrix, pred)
    mae = mean_absolute_error(test_matrix, pred)
    rmse_list.append(rmse)
    mse_list.append(mse)
    mae_list.append(mae)

print('Average RMSE:', np.mean(rmse_list))
print('Average MSE:', np.mean(mse_list))
print('Average MAE:', np.mean(mae_list))
#Note: F1 score, Percision and Recall F1 score, precision, and recall are not well-suited for evaluating the relevance of a set of recommended items
#So, its not well suited withh collaborative filtering (not in all cases at least)

Average RMSE: 0.4709549127127028
Average MSE: 0.22246724281761612
Average MAE: 0.35377874155733446


In [ ]:
from sklearn.preprocessing import MinMaxScaler

Content-based Filtering Model:

In [ ]:
#Define each column based on what it represents (Will be used in Hybrid Filtering)
skills_columns = required_skills
experience_column = df_emp_proj_skills['Experience']
rating_column = df_emp_proj_skills['Rating']
projects_column = df_emp_proj_skills['Total_projects']

In [ ]:
#Drop the columns that are not needed
#df_emp_proj_skills.drop(['Area_of_Interest_1', 'Area_of_Interest_2', 'Area_of_Interest_3','Language1','Language2','Language3'], axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Select the relevant columns from df_emp_proj_skills
relevant_columns = ['Experience', 'Rating', 'Total_projects'] + required_skills + ['Domain Name']
#df_content = df_emp_proj_skills[relevant_columns].copy()

In [ ]:
#Make a copy of the df_emp_proj_skills.copy to be used for content-based algo
#Realized I should always make copies the hard way.
df_content = df_emp_proj_skills.copy()

#We applied MinMaxScaler to the selected columns of df_content
scaler = MinMaxScaler()
df_content[['Experience', 'Rating', 'Total_projects'] + skills_columns] = scaler.fit_transform(df_content[['Experience', 'Rating', 'Total_projects'] + skills_columns])

Content-based Filtering Model Recommendations:

In [ ]:
#Compute the cosine similarity matrix
cosine_sim = cosine_similarity(df_content[['Experience', 'Rating', 'Total_projects'] + skills_columns].fillna(0))

In [ ]:
#We computed each employee score based on the avergae of all relevant columns
df_content['score'] = df_content[['Experience', 'Rating', 'Total_projects'] + skills_columns].mean(axis=1)

#Sort employees based on the score (higher scores first)
relevant_employees = df_content.sort_values(by='score', ascending=False)

#Just to make sure we'll drop employees with non-null skills in the target project
relevant_employees = relevant_employees.dropna(subset=skills_columns)

#to be easier to locate emGet the index of the target project in the cosine similarity matrix
target_project_index = relevant_columns.index('Domain Name')

#This line selects the relevant rows and target project column from the cosine similarity matrix
similarities = cosine_sim[relevant_employees.index.values, :][:, target_project_index]

relevant_employees['score'] = similarities
relevant_employees = relevant_employees.sort_values(by='score', ascending=False).head(5)

#Select the Eid and Score columns from recommended employees and merge with df_emp to get their names and other details
recommended_employees = relevant_employees[['Eid', 'score']]
recommended_employees = pd.merge(recommended_employees, df_emp[['Eid', 'Ename']], on='Eid', how='left')

# Display the recommended employees
print(f'Top 5 Employees Recommended for {target_project}:')
print(recommended_employees)

Top 5 Employees Recommended for Java Project:
   Eid     score            Ename
0    9  1.000000   William Berger
1  301  0.999862     Jack Deyoung
2  770  0.999862      Edna Atwell
3  744  0.999637   Karissa Taylor
4  114  0.999637  Kimberly Lozano


In [ ]:
target_project_index

6

Content-based Filtering Model Evaluation:

In [ ]:
#In this function, we created the same exact steps performed on content-based filtering but in a function to be easier to make a function for RMSE
def recommender_algorithm(X, x):
    #We computed each employee score based on the average of all relevant columns
    X['score'] = X[['Experience', 'Rating', 'Total_projects'] + skills_columns].mean(axis=1)

    #Sort employees based on the score (higher scores first)
    relevant_employees = X.sort_values(by='score', ascending=False)

    #Filter employees with non-null skills in the target project
    relevant_employees = relevant_employees.dropna(subset=skills_columns)

    #Get the index of the target project in the cosine similarity matrix
    target_project_index = relevant_columns.index('Domain Name')

    similarities = cosine_sim[relevant_employees.index.values, :][:, target_project_index]

    relevant_employees['score'] = similarities
    relevant_employees = relevant_employees.sort_values(by='score', ascending=False).head(5)

    #Return the recommended employees
    return relevant_employees

In [ ]:
#This function is generated
#Its supposed to calculate RMSE based on relevant_employees
def measure_rmse_mse_mae(x, relevant_employees):
    #Get the list of recommended employee ids
    recommended_ids = relevant_employees['Eid'].tolist()
    
    #Get the list of actual employee skills
    actual_skills = x[skills_columns].dropna().values
    
    #Compute the cosine similarity between the actual skills and the skills of the recommended employees
    recommended_skills = df_content.loc[recommended_ids, skills_columns].values
    similarities = cosine_similarity(recommended_skills, actual_skills.reshape(1, -1))
    
    #Compute the MSE, RMSE and MAE between the similarities and 1
    mse = np.mean((similarities - 1)**2)
    mae = np.mean(np.abs(similarities - 1))
    rmse = np.sqrt(np.mean((similarities - 1)**2))
    
    return rmse, mse, mae

In [ ]:
X = df_content.drop('score', axis=1).copy()
rmse_list = []
mse_list = []  
mae_list = []  
for i in range(len(X)):
    #Remove row i (employee) from X
    x_i = X.iloc[i]
    X_i = X.drop(i)
    relevant_employees = recommender_algorithm(X_i, x_i)
    #Calculate the RMSE and MSE of the recommendation
    rmse, mse, mae = measure_rmse_mse_mae(x_i, relevant_employees)
    rmse_list.append(rmse)
    mse_list.append(mse)  
    mae_list.append(mae)

#Compute the average RMSE, MSE and MAE
avg_rmse = sum(rmse_list) / len(rmse_list)
avg_mse = sum(mse_list) / len(mse_list)
avg_mae = sum(mae_list) / len(mae_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MSE: {avg_mse}')
print(f'Average MAE: {avg_mae}')

Average RMSE: 0.720337764796717
Average MSE: 0.6563991323210424
Average MAE: 0.6563991323210424


Hybrid Filtering Model:

In [ ]:
#Apply content-based filtering and store them in content_recs
df_content = df_emp_proj_skills.copy()
scaler = MinMaxScaler()
df_content[['Experience', 'Rating', 'Total_projects'] + skills_columns] = scaler.fit_transform(df_content[['Experience', 'Rating', 'Total_projects'] + skills_columns])
cosine_sim = cosine_similarity(df_content[['Experience', 'Rating', 'Total_projects'] + skills_columns].fillna(0))
df_content['score'] = df_content[['Experience', 'Rating', 'Total_projects'] + skills_columns].mean(axis=1)
relevant_employees_content = df_content.sort_values(by='score', ascending=False).dropna(subset=skills_columns)
target_project_index = relevant_columns.index('Domain Name')
similarities = cosine_sim[relevant_employees_content.index.values, :][:, target_project_index]
relevant_employees_content['score'] = similarities
relevant_employees_content = relevant_employees.sort_values(by='score', ascending=False).head(5)[['Ename', 'score']]
content_recs = list(relevant_employees_content.Ename)
#We tried storing them using Eid but it got complicated

In [ ]:
#Use collaborative filtering to get baseline recommendations
testset = data.build_full_trainset().build_testset()
predictions = algo.test(testset)
prediction_df = pd.DataFrame(predictions, columns=['userID', 'itemID', 'actual', 'cf_predictions', 'details'])
baseline_recs = list(prediction_df.groupby('userID').apply(lambda x: x.nlargest(5, 'cf_predictions')).itemID)
#By using collaborative filtering as a baseline, we can compare the performance of other recommendation techniques

Hybrid Filtering Model Recommendations:

In [ ]:
#We will made predictions using both Collaborative and Content-based filtering and add the results for a hybrid filtering approach
#Combine the results from both approaches
hybrid_recs = list(set(baseline_recs+content_recs))

#Merge with df_emp to include employee details
recommendations = pd.merge(df_emp[['Eid', 'Ename']], relevant_employees_content[['Ename', 'score']], on='Ename', how='left')
recommendations['score'] = recommendations['score'].fillna(0)
recommendations['hybrid_score'] = recommendations['score']
recommendations.loc[recommendations['Ename'].isin(hybrid_recs), 'hybrid_score'] += 1
#hybrid_score represents a combined score for each employee based on both collaborative and content-based filtering

#Display the hybrid recommendations
print(f'Top 5 Employees Recommended for {target_project}:')
print(recommendations[['Ename', 'hybrid_score']].sort_values(by='hybrid_score', ascending=False).head(5))


Top 5 Employees Recommended for Java Project:
              Ename  hybrid_score
8    William Berger      2.000000
300    Jack Deyoung      1.999862
769     Edna Atwell      1.999862
902       Eva Notti      1.999637
743  Karissa Taylor      1.999637


Hybrid Filtering Model Evaluation:

In [ ]:
#Convert the train dataframe into a surprise dataset
train_df = pd.melt(df_skills, id_vars=['Eid'], value_vars=required_skills, var_name='Skills', value_name='Rating')
data = Dataset.load_from_df(train_df[['Eid', 'Skills', 'Rating']], reader)

#Use K-fold cross-validation to evaluate the algorithm
kf = KFold(n_splits=5, random_state=42)

#Initialize SVD algorithm with the global mean of the training set as the default value for missing ratings
algo_cf = SVD(biased=False, verbose=False, random_state=42)

<ipython-input-45-cf96e936f8cb>:2: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  train_df = pd.melt(df_skills, id_vars=['Eid'], value_vars=required_skills, var_name='Skills', value_name='Rating')


In [ ]:
#Make a copy of the df_emp_proj_skills to use for analysis
df_content = df_emp_proj_skills.copy()

#Normalize the values using MinMax
scaler = MinMaxScaler()
df_content[['Experience', 'Rating', 'Total_projects'] + skills_columns] = scaler.fit_transform(df_content[['Experience', 'Rating', 'Total_projects'] + skills_columns])
df_content['score'] = df_content[['Experience', 'Rating', 'Total_projects'] + skills_columns].mean(axis=1)
relevant_employees_cb = df_content.sort_values(by='score', ascending=False).dropna(subset=skills_columns)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse_list = []
mse_list = []
mae_list = []

#Split the data into training and test sets using k-fold cross-validation
for trainset, testset in kf.split(data):
    algo_cf.fit(trainset)
    test_pred = algo_cf.test(testset)
    
    #Convert the test set into the expected format (user ID, item ID, true rating) -> (user ID, item ID)
    testset_uid_iid = [(t[0], t[1]) for t in testset]
    
    #Then, compute collaborative filtering predictions
    cf_pred = [algo_cf.predict(uid, iid).est for (uid, iid) in testset_uid_iid]
    
    #Now the content-based predictions for the test set
    cb_pred = []
    for (uid, iid) in testset_uid_iid:
        try:
            #Extract the employee's skills, experience, rating, and projects from the content-based data
            employee_skills = relevant_employees_cb.loc[uid, skills_columns].values.reshape(1, -1)
            employee_experience = experience_column[experience_column.index == uid].values
            employee_rating = rating_column[rating_column.index == uid].values
            employee_projects = projects_column[projects_column.index == uid].values
            
            #Scale and combine the features to create a feature vector for the employee
            employee_features = scaler.transform(np.hstack([employee_skills, employee_experience, employee_rating, employee_projects]).reshape(1, -1))
            
            #Compute the content-based prediction as the average of the feature vector
            cb_pred.append(employee_features.mean())
            
        except:
            #If there is no skill information available for an employee, use the global average from the collaborative filtering algorithm
            cb_pred.append(np.mean(cf_pred))
    
    #Compute the hybrid predictions as the average of the collaborative and content-based predictions
    hybrid_pred = [(cf + cb) / 2 for (cf, cb) in zip(cf_pred, cb_pred)]
    
    #Convert the test set ratings into a list
    testset_ratings = [t.r_ui for t in test_pred]
    rmse = sqrt(mean_squared_error(testset_ratings, hybrid_pred))
    mse = mean_squared_error(testset_ratings, hybrid_pred)
    mae = mean_absolute_error(testset_ratings, hybrid_pred)
    
    rmse_list.append(rmse)
    mse_list.append(mse)
    mae_list.append(mae)

#Compute the average RMSE, MSE, and MAE
avg_rmse = np.mean(rmse_list)
avg_mse = np.mean(mse_list)
avg_mae = np.mean(mae_list)

print('Average RMSE:', avg_rmse)
print('Average MSE:', avg_mse)
print('Average MAE:', avg_mae)


Average RMSE: 0.7706130519307783
Average MSE: 0.5944887765254617
Average MAE: 0.5671869712504939
